In [1]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    ca-certificates-2019.6.16  |       hecc5488_0         145 KB  conda-forge
    certifi-2019.6.16          |           py36_1         149 KB  conda-forge
    openssl-1.1.1c             |       h516909a_0         2.1 MB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.49-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-

In [2]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

In [3]:
# The code was removed by Watson Studio for sharing.

In [4]:
#Hard imposed limit of 50
LIMIT = 50

In [5]:
print ('What version of the API would you like to run in YYYYMMDD format?')
VERSION = input ()
print ("Where would you like to search?")
address = input ()
print('What would you like to search for')
search_query = input ()
print('How far away in meters woud you like the radius to be?')
radius = input ()
geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
#print(latitude, longitude)
#print(search_query + ' .... OK!')

What version of the API would you like to run in YYYYMMDD format?
20190701
Where would you like to search?
Sydney, Australia
What would you like to search for
Mc Donald's
How far away in meters woud you like the radius to be?
100000


In [6]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&limit={}&radius={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query,LIMIT,radius)
#print (url)

results = requests.get(url).json()
#print (results)

# assign relevant part of JSON to venues
venues = results['response']['venues']

In [7]:
# tranform venues into a dataframe
dataframe = json_normalize(venues)
print (dataframe)

                                           categories  hasPerk  \
0   [{'id': '4bf58dd8d48988d16e941735', 'name': 'F...    False   
1   [{'id': '4bf58dd8d48988d16e941735', 'name': 'F...    False   
2   [{'id': '4bf58dd8d48988d16e941735', 'name': 'F...    False   
3   [{'id': '4bf58dd8d48988d16e941735', 'name': 'F...    False   
4   [{'id': '4bf58dd8d48988d16e941735', 'name': 'F...    False   
5   [{'id': '4bf58dd8d48988d16e941735', 'name': 'F...    False   
6   [{'id': '4bf58dd8d48988d16e941735', 'name': 'F...    False   
7   [{'id': '4bf58dd8d48988d16e941735', 'name': 'F...    False   
8   [{'id': '4bf58dd8d48988d16e941735', 'name': 'F...    False   
9   [{'id': '4bf58dd8d48988d16e941735', 'name': 'F...    False   
10  [{'id': '4bf58dd8d48988d16e941735', 'name': 'F...    False   
11  [{'id': '4bf58dd8d48988d16e941735', 'name': 'F...    False   
12  [{'id': '4bf58dd8d48988d16e941735', 'name': 'F...    False   
13  [{'id': '4bf58dd8d48988d16e941735', 'name': 'F...    False   
14  [{'id'

In [8]:
maccas_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the centre location

# add a red circle marker to represent the Conrad Hotel
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='blue',
    popup='Centre',
    fill = True,
    fill_color = 'blue',
    fill_opacity = 0.6
).add_to(maccas_map)

# add the Maccas restaurants as blue circle markers
dataframeSummary = dataframe[['location.lat','location.lng', 'location.city']]
#print(dataframeSummary.dtypes)
#print (dataframeSummary.head())

for index, row in dataframeSummary.iterrows():
    folium.features.CircleMarker(
        [row['location.lat'], row['location.lng']],
        radius=5,
        color='red',
        fill = True,
        fill_color='red',
        fill_opacity=0.6
    ).add_to(maccas_map)
    
# display map
maccas_map


In [9]:
#dataframeSummary = dataframe.groupby('location.city').count()['id']
dataframeSummary = dataframe[['location.lat','location.lng', 'location.distance']]
print (dataframeSummary.head())
#print (dataframe[['location.lat','location.lng', 'location.city']])

   location.lat  location.lng  location.distance
0    -33.861800    151.210400                957
1    -33.873717    151.223648               2206
2    -33.883858    151.202521               3479
3    -33.869308    151.207016               1834
4    -33.875388    151.206632               2463


In [17]:
# set number of clusters
kclusters = 3

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(dataframeSummary)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

# add clustering labels
#dataframeSummary.insert(0, 'Cluster Labels', kmeans.labels_)

dataframe_merged = dataframe[['location.city']]

#print (dataframe.head())
print (dataframeSummary.head())
print (kmeans)

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
dataframe_merged = pd.concat([dataframe_merged, dataframeSummary], axis=1, ignore_index=False)

dataframe_merged.head() # check the last columns!

   Cluster Labels  location.lat  location.lng  location.distance
0               2    -33.861800    151.210400                957
1               2    -33.873717    151.223648               2206
2               2    -33.883858    151.202521               3479
3               2    -33.869308    151.207016               1834
4               2    -33.875388    151.206632               2463
KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=3, n_init=10, n_jobs=None, precompute_distances='auto',
    random_state=0, tol=0.0001, verbose=0)


,location.city,Cluster Labels,location.lat,location.lng,location.distance
0,Sydney,2,-33.861800,151.210400,957
1,Kings Cross,2,-33.873717,151.223648,2206
2,Broadway,2,-33.883858,151.202521,3479
3,Sydney,2,-33.869308,151.207016,1834
4,Sydney,2,-33.875388,151.206632,2463


In [20]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(dataframe_merged['location.lat'], dataframe_merged['location.lng'], dataframe_merged['location.city'], dataframe_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters